In [ ]:
# First cell: Set env + install (run once)
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'  # fragmentation fix

!pip install torch matplotlib numpy

# Second cell: The sim code (no OOM/assertion — safe params)
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.amp
from torch.utils.checkpoint import checkpoint
import numpy as np
from contextlib import nullcontext
import math

torch.cuda.empty_cache()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# CONFIG – optimized for no OOM + H100 speed (fast epochs)
triality = 3
dim = 384  # divisible by 3 and 8
latent_dim = 8
image_size = 64  # reduced seq_len ~12288
batch_size = 16  # reduced
epochs = 20000
lr = 5e-5
use_amp = True
use_checkpoint = True

# Synthetic lunar proxy (crater patterns + dust noise/occlusion)
images = []
for b in range(batch_size):
    frame = torch.randn(3, image_size, image_size, device=device) * 0.05 + 0.5
    frame = torch.clamp(frame, 0, 1)

    y, x = torch.meshgrid(torch.arange(image_size, device=device), torch.arange(image_size, device=device), indexing='ij')
    for _ in range(15):
        cx = torch.randint(0, image_size, (1,), device=device).item()
        cy = torch.randint(0, image_size, (1,), device=device).item()
        radius = torch.randint(4, 12, (1,)).item()
        dist = ((x - cx)**2 + (y - cy)**2)
        crater = (dist < radius**2)
        frame[:, crater] *= 0.5

    frame += torch.randn_like(frame) * 0.1
    frame = torch.clamp(frame, 0, 1)

    images.append(frame)

images = torch.stack(images).to(device)

# Flatten + project to dim
real_data = images.flatten(2).permute(0, 2, 1)
real_data = F.pad(real_data, (0, dim - real_data.shape[-1]))

# Apply masking (40–70% missing pixels — dust/occlusion)
missing_rate = torch.linspace(0.4, 0.7, batch_size, device=device).view(batch_size, 1, 1)
mask = torch.rand_like(real_data) < missing_rate
real_data[mask] = 0

target = real_data.clone()

# E8 roots – precompute
def get_e8_roots():
    roots = []
    for i in range(8):
        for j in range(i+1, 8):
            for signs in [(1,1), (1,-1), (-1,1), (-1,-1)]:
                v = torch.zeros(8)
                v[i] = signs[0]; v[j] = signs[1]
                roots.append(v); roots.append(-v)
    for signs in range(1 << 8):
        v = torch.tensor([(1 if (signs & (1<<k)) else -1) for k in range(8)], dtype=torch.float32) * 0.5
        if bin(signs).count('1') % 2 == 0:
            roots.append(v); roots.append(-v)
    roots = torch.stack(roots[:240])
    return roots / roots.norm(dim=-1, keepdim=True)

e8_roots = get_e8_roots().to(device)

# Triality Cycle Block (detached pump scalar)
class LunarCycleBlock(nn.Module):
    def __init__(self):
        super().__init__()
        self.proj = nn.Linear(latent_dim, dim // triality, bias=False)
        self.register_buffer('roots', e8_roots)

    def forward(self, x, step):
        pos_emb = self.roots[torch.arange(x.shape[1], device=device) % 240]
        low_dim = self.proj(pos_emb)
        emb = low_dim.repeat(1, triality)
        with torch.no_grad():
            pump_scalar = 0.8 * math.sin(step * 0.006 * 2 * math.pi)
        pump = torch.full((1, x.shape[1], 1), pump_scalar, device=device)
        emb_broadcast = emb.unsqueeze(0)
        x_rot1 = x * (emb_broadcast.cos() + pump)
        x_rot2 = torch.roll(x_rot1, shifts=1, dims=1) * emb_broadcast.sin()
        x_rot3 = torch.roll(x_rot2, shifts=1, dims=1) * emb_broadcast.cos()
        fused = (x_rot1 + x_rot2 + x_rot3) / triality
        return fused

# Dummy cycle for ablation
class DummyCycle(nn.Module):
    def forward(self, x, step=None):
        return x

# Model with ablation support (safe num_heads)
class E8LunarFusion(nn.Module):
    def __init__(self, depth=16, use_triality=True):
        super().__init__()
        self.use_triality = use_triality
        self.cycle = LunarCycleBlock() if use_triality else DummyCycle()
        num_heads = triality if use_triality else 8  # 3 or 8, both divide 384
        self.layers = nn.ModuleList([nn.MultiheadAttention(dim, num_heads, batch_first=True) for _ in range(depth)])
        self.norm = nn.LayerNorm(dim)
        self.head = nn.Linear(dim, dim)

    def forward(self, x, step):
        x = self.cycle(x, step)
        for layer in self.layers:
            if use_checkpoint:
                attn, _ = checkpoint(layer, x, x, x, use_reentrant=False)
            else:
                attn, _ = layer(x, x, x)
            x = x + self.norm(attn)
        return x

# Models
model = E8LunarFusion(use_triality=True).to(device)
model_ablation = E8LunarFusion(use_triality=False).to(device)

opt = torch.optim.AdamW(model.parameters(), lr=lr)
scaler = torch.amp.GradScaler('cuda') if use_amp else nullcontext()

opt_ablation = torch.optim.AdamW(model_ablation.parameters(), lr=lr)
scaler_ablation = torch.amp.GradScaler('cuda') if use_amp else nullcontext()

loss_fn = nn.MSELoss()

loss_hist = []
loss_abl_hist = []

for epoch in range(epochs):
    opt.zero_grad(set_to_none=True)
    opt_ablation.zero_grad(set_to_none=True)

    with torch.amp.autocast(device_type='cuda', dtype=torch.float16) if use_amp else nullcontext():
        recon = model(real_data, epoch)
        loss = loss_fn(recon, target)

        recon_abl = model_ablation(real_data, epoch)
        loss_abl = loss_fn(recon_abl, target)

    scaler.scale(loss).backward() if use_amp else loss.backward()
    scaler.unscale_(opt) if use_amp else None
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1e6)
    scaler.step(opt) if use_amp else opt.step()
    scaler.update() if use_amp else None

    scaler_ablation.scale(loss_abl).backward() if use_amp else loss_abl.backward()
    scaler_ablation.unscale_(opt_ablation) if use_amp else None
    torch.nn.utils.clip_grad_norm_(model_ablation.parameters(), 1e6)
    scaler_ablation.step(opt_ablation) if use_amp else opt_ablation.step()
    scaler_ablation.update() if use_amp else None

    loss_hist.append(loss.item())
    loss_abl_hist.append(loss_abl.item())

    if epoch % 500 == 0:
        print(f"Epoch {epoch} | Triality Loss {loss.item():.6f} | Ablation Loss {loss_abl.item():.6f}")

# Final Sigma Test (Triality advantage over ablation)
triality_mean = np.mean(loss_hist)
abl_mean = np.mean(loss_abl_hist)
std = np.std(loss_hist + loss_abl_hist)
sigma = (abl_mean - triality_mean) / std if std > 0 else 0

print(f"Final Sigma (Triality vs Ablation): {sigma:.2f} (higher = triality advantage)")

print("Sim complete — epochs + sigma test done")


[notice] A new release of pip is available: 24.2 -> 26.0
[notice] To update, run: python -m pip install --upgrade pip
Using device: cuda


/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]


Epoch 0 | Triality Loss 17.599632 | Ablation Loss 14.546055
Epoch 500 | Triality Loss 0.003088 | Ablation Loss 0.001170
Epoch 1000 | Triality Loss 0.001400 | Ablation Loss 0.002272
Epoch 1500 | Triality Loss 0.000903 | Ablation Loss 0.002194
Epoch 2000 | Triality Loss 0.000509 | Ablation Loss 0.000486
Epoch 2500 | Triality Loss 0.000577 | Ablation Loss 0.001691
Epoch 3000 | Triality Loss 0.000495 | Ablation Loss 0.000587
Epoch 3500 | Triality Loss 0.000311 | Ablation Loss 0.000921
Epoch 4000 | Triality Loss 0.000387 | Ablation Loss 0.001630
Epoch 4500 | Triality Loss 0.000316 | Ablation Loss 0.000259
Epoch 5000 | Triality Loss 0.000250 | Ablation Loss 0.000454
Epoch 5500 | Triality Loss 0.000240 | Ablation Loss 0.000324
Epoch 6000 | Triality Loss 0.000328 | Ablation Loss 0.000237
Epoch 6500 | Triality Loss 0.000282 | Ablation Loss 0.000450
